In [4]:
from __future__ import annotations
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4" 

import json
from pathlib import Path
from typing import List, Dict

import faiss  # type: ignore
import numpy as np
import torch
from FlagEmbedding import BGEM3FlagModel
from BGERetriever import BGERetriever

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
emb_model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=True)
retriever = BGERetriever(
    embedding_model=emb_model,
    index_path="/local00/student/shakya/openmath_bge-m3_hnsw_index",
    metadata_path="/local00/student/shakya/openmath_bge-m3_metadata.jsonl",
    device=device,
)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
docs = retriever.search("pythagorean theorem", k=200)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
pairs = [["pythagorean theorem", d["solution_chunk"]] for d in docs]
scores = retriever.model.compute_score(pairs)["colbert"]
for d, s in zip(docs, scores):
    d["rerank_score"] = s
docs = sorted(docs, key=lambda d: d["rerank_score"], reverse=True)[:10]

In [19]:
docs

[{'id': 1942037000,
  'score': 0.57972115,
  'solution_chunk': 'In a right triangle, the Pythagorean theorem states that the sum of the squares of the lengths of the two shorter sides is equal to the square of the length of the hypotenuse.',
  'problem': 'If the lengths of two sides of a right triangle are 5 and 12 units, what is the least possible length, in units, of the third side? Express your answer in simplest radical form.',
  'rerank_score': 0.7772959470748901},
 {'id': 8408518000,
  'score': 0.58767045,
  'solution_chunk': 'The Pythagorean theorem states that in a right-angled triangle, the square of the hypotenuse side is equal to the sum of squares of the other two sides.',
  'problem': 'Given a quadratic equation in the form of ax^2 + bx + c = 0, determine the value of x that satisfies the equation and also forms the hypotenuse of a right-angled triangle with legs of lengths 5 and 12. If no such value of x exists, determine the closest approximation to the solution.',
  're

In [20]:
retrieved_context = ""
for i, (h_list) in enumerate((docs), 1):
    retrieved_context += f"question {i}:\n"
    retrieved_context += h_list.get("problem")
    retrieved_context += "\n"
    retrieved_context += h_list.get("solution_chunk") or h_list.get("text")
    retrieved_context += "\n"

In [21]:
print(retrieved_context)

question 1:
If the lengths of two sides of a right triangle are 5 and 12 units, what is the least possible length, in units, of the third side? Express your answer in simplest radical form.
In a right triangle, the Pythagorean theorem states that the sum of the squares of the lengths of the two shorter sides is equal to the square of the length of the hypotenuse.
question 2:
Given a quadratic equation in the form of ax^2 + bx + c = 0, determine the value of x that satisfies the equation and also forms the hypotenuse of a right-angled triangle with legs of lengths 5 and 12. If no such value of x exists, determine the closest approximation to the solution.
The Pythagorean theorem states that in a right-angled triangle, the square of the hypotenuse side is equal to the sum of squares of the other two sides.
question 3:
The sides of a right triangle have lengths $a, a+2$, and $2a$. Find the ratio of the lengths of the sides of the triangle in simplest form.
The Pythagorean Theorem states t

In [16]:
init("/local00/student/shakya/openmath_bge-m3_hnsw_index",
     "/local00/student/shakya/openmath_bge-m3_metadata.jsonl")

⏳  Loading Faiss index …
✅  index loaded: 22,392,715 vectors, dim=1024
⏳  Loading metadata …
✅  22,392,715 meta rows
⏳  Loading BGE‑m3 embedder …


Fetching 30 files:   0%|          | 0/30 [00:05<?, ?it/s]

⏳  Loading reranker (cross‑encoder) …
🎉  init() complete. Ready to search.


In [33]:
hits = search("pythagorean theorem")
hits

[{'id': 1259277004,
  'row_id': '1259277',
  'chunk_id': '4',
  'problem': 'Let $EFGH$ be an isosceles trapezoid, whose dimensions are $EF = 8, FG=6=HE,$and $GH=5.$ Draw circles of radius 4 centered at $E$ and $F,$ and circles of radius 3 centered at $G$ and $H.$ A circle contained within the trapezoid is tangent to all four of these circles. Its radius is $\\frac{m\\sqrt{n}-k}{p},$ where $k, m, n,$ and $p$ are positive integers, $n$ is not divisible by the square of any prime, and $k$ and $p$ are relatively prime. Find $k+m+n+p.$',
  'solution_chunk': '## Step 3: Apply the Pythagorean Theorem The Pythagorean Theorem states that in a right-angled triangle, the square of the length of the hypotenuse is equal to the sum of the squares of the lengths of the other two sides. We can apply this theorem to the right-angled triangles formed by the lines $AO$ and $DO$ and the radii of the circles.',
  'expected_answer': '134',
  'problem_from': 'augmented_math',
  'rerank_score': 6.05859375,
  

In [39]:
retrieved_context = ""
for i, (h_list) in enumerate((hits), 1):
    retrieved_context += f"question {i}:\nResponse:"
    retrieved_context += h_list.get("problem")
    retrieved_context += "\n"
    retrieved_context += h_list.get("solution_chunk") or h_list.get("text")
    retrieved_context += "\n"

retrieved_context 

'question 1:\nResponse:Let $EFGH$ be an isosceles trapezoid, whose dimensions are $EF = 8, FG=6=HE,$and $GH=5.$ Draw circles of radius 4 centered at $E$ and $F,$ and circles of radius 3 centered at $G$ and $H.$ A circle contained within the trapezoid is tangent to all four of these circles. Its radius is $\\frac{m\\sqrt{n}-k}{p},$ where $k, m, n,$ and $p$ are positive integers, $n$ is not divisible by the square of any prime, and $k$ and $p$ are relatively prime. Find $k+m+n+p.$\n## Step 3: Apply the Pythagorean Theorem The Pythagorean Theorem states that in a right-angled triangle, the square of the length of the hypotenuse is equal to the sum of the squares of the lengths of the other two sides. We can apply this theorem to the right-angled triangles formed by the lines $AO$ and $DO$ and the radii of the circles.\nquestion 2:\nResponse:The legs $PQ$ and $PR$ of a right triangle $PQR$ have lengths which are positive whole numbers. Given that $PQ = k \\cdot PR$ for some positive intege